<a href="https://colab.research.google.com/github/Chris300127/basil_leaf_project/blob/master/Basil_Leaf_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')
import time
import cv2
import numpy as np 
import sys
from google.colab.patches import cv2_imshow

#ML libraries
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


#### Number of images for each dataset
- Healthy Basil: 148
- Healthy Jamun: 279
- Diseased Jamun: 345

In [0]:
#load images from google drive

def load_image(picNum=1, leaf_type='Basil',healthy=True, scale=0.1):
  if picNum < 10:
    picNum = f'00{picNum}'
  elif picNum < 100:
    picNum = f'0{picNum}'
  
  #leaf type
  if leaf_type == 'Basil':
    path=f'/content/drive/Shared drives/Senior Design/COEN/Photos/Online_Dataset/Basil (P8)/healthy/0008_0{picNum}.JPG'
  elif leaf_type == 'Jamun':
    if healthy:
      path=f'/content/drive/Shared drives/Senior Design/COEN/Photos/Online_Dataset/Jamun (P5)/healthy/0005_0{picNum}.JPG'
    else:
      path=f'/content/drive/Shared drives/Senior Design/COEN/Photos/Online_Dataset/Jamun (P5)/diseased/0017_0{picNum}.JPG'
  #read in and rescale   
   #scale factor for each photo
  img = cv2.imread(path)
  width = int(img.shape[1] * scale)
  height = int(img.shape[0] * scale)
  dim=(width,height)

  # resize image
  return cv2.resize(img,dim,interpolation=cv2.INTER_AREA)

print("Done")

Done


// Scaling it to the same size

*   List item
*   List item


// CNN (images as input)

//SVM, PCA or LDA w/ kmean

//https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html


In [0]:
#ML: Linear Discriminant Analysis
#================================

#load training data
#-------------------
training_pics = []
training_health = []

# will load 30 healthy and 30 diseased leaves into memory
for i in range (1,31):
  img = load_image(i,'Jamun',healthy=True)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  training_pics.append(img.flatten())
  training_health.append('healthy')
  img = load_image(i,'Jamun',healthy=False)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  training_pics.append(img.flatten())
  training_health.append('diseased')

training_pics = np.array(training_pics)
training_health = np.array(training_health)
print("Training Images Loaded")
# print(training_pics.shape,training_health.shape)


Training Images Loaded


In [0]:
# Create LDA Model from training images
#------------------------------------
lda = LinearDiscriminantAnalysis(solver="svd", store_covariance=False)
lda.fit(training_pics, training_health)
print("LDA Model Created")

LDA Model Created


In [0]:
#load testing images
#-------------------
testing_pics = []
testing_health = []

def load_testing(first_image, end_image):
  global testing_pics, testing_health
  testing_pics = []
  testing_health = []

  for j in range(first_image, end_image):
    img = load_image(i,'Jamun',healthy=True)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    testing_pics.append(img.flatten())
    testing_health.append('healthy')
    img = load_image(i,'Jamun',healthy=False)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    testing_pics.append(img.flatten())
    testing_health.append('diseased')

  testing_pics = np.array(testing_pics)
  testing_health = np.array(testing_health)
  print("Testing Images Loaded")
  # print(testing_pics.shape,testing_health.shape)


In [0]:
# Scoring data
#-------------
def score():
  #score model with training original training data
  score1 = lda.score(training_pics, training_health)
  print("Training score:",score1)

  #score model with new testin data
  score2 = lda.score(testing_pics, testing_health)
  print("Testing score:",score2)


In [0]:
load_testing(230,245)
score()

Testing Images Loaded
Training score: 0.9166666666666666
Testing score: 1.0


In [0]:
# good_leaves = []  #will contain coordinates of good leaves


def pixelsToInches(pixel):
    return pixel * (18.6/640)


def findLeaf(picNum=1):
  sample = load_image(picNum)
  gray = cv2.cvtColor(sample, cv2.COLOR_BGR2GRAY)
  blurred = cv2.GaussianBlur(gray,(5,5),cv2.BORDER_DEFAULT)
  canny = cv2.Canny(blurred,50,100)
  kernel = np.ones((3,3),np.uint8)
  dilate = cv2.dilate(canny, kernel, iterations=1)
  cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  cnts = cnts[0] if len(cnts)==2 else cnts[1] #gets contours from tuple return value (which may be tuple of 2 or three depending on cv2 version)
  # print("Number of leaves: ", len(cnts))
  # LEAF_MUTEX.acquire()
  display_image = sample #which image to display

  max_area = 0
  for c in cnts:
    # c = cnts[0]
    b_box = cv2.boundingRect(c)
    #find largest b_box area
    a = b_box[2]*b_box[3] #current bounding box area
    if a > max_area:
      max_area = a
      x,y,w,h = b_box

    
  
  
  
  cv2.rectangle(display_image, (x,y), (x+w, y+h), (36,255,12), 2)  #numbers specify color and width 
  # ROI = original[y:y+h, x:x+w]
  r = 5
  t = 1
  cx = x + int(w / 2)
  cy = y + int(h / 2)
  cv2.circle(display_image, (cx, cy), r, (255, 0, 0), thickness=t)
  
  # print(pixelsToInches(x+cx),pixelsToInches(y+cy))
  
  if len(cnts)>0: #change from 0 to 1 to see potential errors
    print("\nImage",picNum,"- leaf found at:",cx,cy)
    cv2_imshow(display_image)
 


  # Run extraction and ML
  # should return bool if leaf is good or not good
  # good = True
  # if(good):
  #   #insert into goodleaves
  #   good_leaves.append((cx, cy))
  # else:
  #   pass
  #   # discardLeaves()
  # good_leaves.sort()

  # LEAF_MUTEX.release()
  # while True:
  #   if cv2.waitKey(1)==27:  # esc key
  #     break
  # cap.release()
  # cv2.destroyAllWindows()


for i in range(1,280):
  findLeaf(i)
print("\n\nDone\n\n")


In [0]:

for i in range(1,149):
  findLeaf(i)


###Basil Leaves
- bounding boxes are a little off on image 45 & 70
- all others are good


###Jamun Leaves
- Bounding box is off for images 14,20, & 198
